In [1]:
# Import
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.integrate import quad


In [2]:
# Parameters
n = 30
C = 4
meanT = 40
meanM = 15
meanM = meanM - 1
VarM = meanM**2
oneItemST = 3

# Parameters of the LOGNORMAL distribution
mu = math.log(meanM**2/math.sqrt(VarM+meanM**2))
sigma = math.sqrt(math.log((VarM/meanM**2)+1))

# The number of items in a customer's cart is a LOGNORMAL random variable
def lognormCDF(x_, mu_, sigma_):
    return lognorm.cdf(x_, s=sigma_, scale=math.exp(mu_))

def lognormPDF(x_, mu_, sigma_):
    return lognorm.pdf(x_, s=sigma_, scale=math.exp(mu_))

# Factors
p = 0.5
K = 8

# CSV name
csv_name = "VTM_Lognormal"

In [3]:
print("mu = ", mu)
print("sigma = ", sigma)

mu =  2.292483739335286
sigma =  0.8325546111576977


In [4]:
CSV = pd.read_csv(f"{csv_name}.csv").fillna(0)
CSV = CSV[CSV["type"] == "scalar"]

# NQ: Mean Number of Customers in Queue [Queues]
Exp_NQ = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_NQ = Exp_NQ[Exp_NQ["name"] == "queueLength:timeavg"]

# RHO: Mean Utilization [Queues]
Exp_RHO = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_RHO = Exp_RHO[Exp_RHO["name"] == "busy:timeavg"]

# S: Mean Service Time (t_S) [LocalSink] 
Exp_S = CSV[CSV["module"].str.contains("SuperMarket.checkout.localSink")]
Exp_S = Exp_S[Exp_S["name"] == "totalServiceTime:mean"]

# W: Mean Waiting Time [Queues]
Exp_W = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_W = Exp_W[Exp_W["name"] == "queueingTime:mean"]


# Dictionary for containing all the DataFrames
CSV_Dict = {
    "NQ": Exp_NQ,
    "RHO": Exp_RHO,
    "S": Exp_S,
    "W": Exp_W
}


PI = {}

Q = []
for i in range(C):
    Q.append(f"Q{i}")

for key in CSV_Dict:
    PI[key] = pd.DataFrame(columns=Q)
    for i in range(C):
        temp = []
        temp = [CSV_Dict[key]["value"].values[j*C + i] for j in range(n)]
        string = f"Q{i}"
        temp = list(map(float, temp))
        PI[key][string] = temp
   

In [5]:
PI_TABLE = {}

for key in PI:
    PI_TABLE[key] = pd.DataFrame(columns=["Theoretical", "Mean", "CI - LB", "CI - UB"])
    
    # Mean Value of the "n" repetitions for each Queue
    PI_TABLE[key]["Mean"] = PI[key].mean()

    # CI 95%
    # Lower Bound
    PI_TABLE[key]["CI - LB"] = PI[key].mean() - (1.96 * (PI[key].std() / math.sqrt(n)))
    # Upper Bound
    PI_TABLE[key]["CI - UB"] = PI[key].mean() + (1.96 * (PI[key].std() / math.sqrt(n)))



In [6]:
# for key in PI_TABLE:
#     print(f"[{key}]:")
#     print(PI_TABLE[key])
#     print("\n")

In [7]:
# Theoretical Formulas and Computations

# g(M) = round(M) + 1 = floor(M+1.5)
def g(x):
    return math.floor(x+1.5)  

def integrand(x):
    return (g(x) * lognormPDF(x, mu, sigma))

def integrand_2(x):
    return ((g(x)**2) * lognormPDF(x, mu, sigma))



# Number of Quick Tills
quickTills = int(p*C)

# Interarrival Rate
lambdaT = 1/meanT

temp = {}

if (quickTills == 0) or quickTills == C:
    # There are no Quick Tills or all the Tills are Quick Tills
    # So the probability alphaK of being routed to a Quick Till is either 0 or 1

    pi_ = 1/C
    lambdaT_ = lambdaT*pi_

    meanST_ = meanM * oneItemST
    lambdaST_ = 1/meanST_
    
    # Utilization
    rho_ = lambdaT_/lambdaST_

    # Mean Number of Customers in M/M/1
    N_ = rho_/(1-rho_)

    # Mean Number of Customers in a Queue
    NQ_ = N_ - rho_

    # Mean Response Time
    R_ = N_/lambdaT_

    # Mean Waiting Time
    W_ = R_ - meanST_


    temp["NQ"] = NQ_
    temp["RHO"] = rho_
    temp["S"] = meanST_
    temp["W"] = W_


    for key in PI_TABLE:
        PI_TABLE[key]["Theoretical"] = [temp[key]] * C

else:
    # There are Quick Tills AND Normal Tills
    # So the probability alphaK of being routed to a Quick Till is 0 < alphaK < 1
    
    # CDF of an EXPONENTIAL distribution
    alphaK = lognormCDF((K-0.5), mu, sigma)

    # --- Quick Tills ---

    # Probability of being routed to a specific Quick Till (i)
    # Inside a subsystem (Quick Tills or Normal Tills) the
    # probability of being routed to a specific Till is the
    # same (i.e. "equallylikely")
    pi_i = alphaK/quickTills
    
    # Interarrival Rate for an individual Quick Till
    lambdaT_i = lambdaT*pi_i
    
    # Mean Number of items in a customer's cart for an individual Quick Till
    meanM_i = (quad(integrand, 0, (K-0.5), limit=200)[0])/alphaK

    # Mean Service Time for an individual Quick Till
    meanST_i = meanM_i * oneItemST


    # Variance = E[X^2] - E[X]^2
    # Variance of M_i
    VarM_i = ((quad(integrand_2, 0, (K-0.5), limit=200)[0])/alphaK) - meanM_i**2

    # Variance of ST_i
    VarST_i = VarM_i * oneItemST**2
    
    
    # Performance Indeces for an individual Quick Till

    rho_i = lambdaT_i * meanST_i
    N_i = rho_i + ((rho_i**2 + lambdaT_i**2 * VarST_i)/(2*(1-rho_i)))
    NQ_i = N_i - rho_i
    R_i = N_i/lambdaT_i
    W_i = R_i - meanST_i
    
    temp["NQ"] = {}
    temp["NQ"]["i"] = NQ_i

    temp["RHO"] = {}
    temp["RHO"]["i"] = rho_i

    temp["S"] = {}
    temp["S"]["i"] = meanST_i

    temp["W"] = {}
    temp["W"]["i"] = W_i


    # --- Normal Tills ---

    # Probability of being routed to a specific Normal Till (j)
    # Inside a subsystem (Quick Tills or Normal Tills) the
    # probability of being routed to a specific Till is the
    # same (i.e. "equallylikely")
    pi_j = (1-alphaK)/(C-quickTills)

    # Interarrival Rate for an individual Normal Till
    lambdaT_j = lambdaT*pi_j

    # Mean Number of items in a customer's cart for an individual Normal Till
    meanM_j = (quad(integrand, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)

    # Mean Service Time for an individual Normal Till
    meanST_j = meanM_j * oneItemST

    # Variance = E[X^2] - E[X]^2
    # Variance of M_j
    VarM_j = ((quad(integrand_2, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)) - meanM_j**2

    # Variance of ST_j
    VarST_j = VarM_j * oneItemST**2
    
    # Performance Indeces for an individual Normal Till

    rho_j = lambdaT_j * meanST_j
    N_j = rho_j + ((rho_j**2 + lambdaT_j**2 * VarST_j)/(2*(1-rho_j)))
    NQ_j = N_j - rho_j
    R_j = N_j/lambdaT_j
    W_j = R_j - meanST_j

    temp["NQ"]["j"] = NQ_j
    temp["RHO"]["j"] = rho_j
    temp["S"]["j"] = meanST_j
    temp["W"]["j"] = W_j
    
    # Adding the Theoretical Values to the PI_TABLE
    for key in PI_TABLE:
        PI_TABLE[key]["Theoretical"] = [temp[key]["i"]] * quickTills + [temp[key]["j"]] * (C-quickTills)

    


C:\Users\giova\AppData\Local\Temp\ipykernel_16196\1932056582.py:124: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  meanM_j = (quad(integrand, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)
C:\Users\giova\AppData\Local\Temp\ipykernel_16196\1932056582.py:131: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from sp

In [8]:
for key in PI_TABLE:
    PI_TABLE[key]["OK"] = [""] * C
    for i in range(C):
        if PI_TABLE[key]["Theoretical"].iloc[i] <= PI_TABLE[key]["CI - UB"].iloc[i] and PI_TABLE[key]["Theoretical"].iloc[i] >= PI_TABLE[key]["CI - LB"].iloc[i]:
            PI_TABLE[key].loc[f"Q{i}", "OK"] = "V"

In [9]:
for key in PI_TABLE:
    print(f"[{key}]:")
    print(PI_TABLE[key])
    print("\n")

[NQ]:
    Theoretical      Mean   CI - LB   CI - UB OK
Q0     0.003569  0.003838  0.003404  0.004271  V
Q1     0.003569  0.003812  0.003512  0.004112  V
Q2     0.351523  0.344048  0.311578  0.376518  V
Q3     0.351523  0.358798  0.334710  0.382886  V


[RHO]:
    Theoretical      Mean   CI - LB   CI - UB OK
Q0     0.077617  0.078654  0.077059  0.080249  V
Q1     0.077617  0.078522  0.076994  0.080050  V
Q2     0.484940  0.480816  0.473381  0.488252  V
Q3     0.484940  0.486472  0.478377  0.494567  V


[S]:
    Theoretical       Mean    CI - LB    CI - UB OK
Q0    16.808667  16.764889  16.668415  16.861364  V
Q1    16.808667  16.833326  16.746425  16.920226  V
Q2    61.522346  61.435091  60.817201  62.052981  V
Q3    61.522346  61.817371  61.173318  62.461425  V


[W]:
    Theoretical       Mean    CI - LB    CI - UB OK
Q0     0.772947   0.814912   0.726942   0.902881  V
Q1     0.772947   0.817020   0.755333   0.878708  V
Q2    44.596290  43.771929  39.764508  47.779350  V
Q3    44.5962

In [10]:
# alphaK is in the denominator because
# it is there to normalize the PDF. Indeed
# calculating the integral of the truncated
# PDF from 0 to K-0.5 gives exactly 1

def integrand_Test(x):
    return lognormPDF(x, mu, sigma)

meanM_i = (quad(integrand_Test, 0, (K-0.5), limit=200)[0])/alphaK
meanM_i

0.9999999999998394

In [11]:
def integrand_Test(x):
    return (math.floor(x+1+0.5)*lognormPDF(x, mu, sigma))

print("Mean(M'') (with Mean(M) = m-1): " + str(quad(integrand_Test, 0, np.inf, limit=200)[0]))


Mean(M'') (with Mean(M) = m-1): 15.000453066265877


C:\Users\giova\AppData\Local\Temp\ipykernel_16196\41242726.py:4: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  print("Mean(M'') (with Mean(M) = m-1): " + str(quad(integrand_Test, 0, np.inf, limit=200)[0]))
